<h1 align="center">Advanced Image Processing and Analysis</h1>
<h3 align="center">ECE 4438B/ECE 9022/ECE 9202B/BIOMED 9519B/BIOPHYS 9519B/CAMI 9519B</h3>
<h4 align="center"><a href="mailto:echen29@uwo.ca?subject=Day 19: Registration basic, translation"> Elvis Chen, PhD, LL</a></h4>
<h4 align="center">Day 19, March 18, 2019</h4>

### Introduction

In this Jupyter Notebook, we will example the implementation details of the registration frameworks within SimpleITK.

When we introduced the concept of similarity metric, all the mental execise we used in class involved the following key concepts:

* a fixed image: the image that does NOT change/move. It is the image we want to use as a reference,
* a moving image: the *mobile* image that we want to **transform**, and
* a transformation: how we want to manipulate the moving image. The mental exercise we went through so far only involved 1D translation (in the x-axis).

Let us look at a simple example in SimpleITK, but involing 2D translation (in both x- and y-axis)

In [ ]:
import SimpleITK as sitk

%run update_path_to_download_script
from downloaddata import fetch_data as fdata

import matplotlib.pyplot as plt

import gui
import registration_gui as rgui
%matplotlib notebook

import numpy as np
import os
OUTPUT_DIR = 'output'

Notice that two additional python modules are used (gui and registration_gui) to facilitate the visualization of registration.

First, let us read 2 images. The registration framework in SimpleITK only work with pixel type of Float16 or Float32, so when we read the images we will force the pixel type at the same time.

In [ ]:
# the fixed image, read the file and store it as sitkFloat32
fixedImage = sitk.ReadImage('BrainProtonDensitySliceBorder20.png', sitk.sitkFloat32 )

In [ ]:
# the moving image, read the file and store it as sitkFloat32
movingImage = sitk.ReadImage('BrainProtonDensitySliceShifted13x17y.png', sitk.sitkFloat32 )

Next, we define a simple utility function to help us to visualize these two images side by side. For 3D volumes, we can use utilitiy functions from gui and registration_gui.

In [ ]:
# define an utility function to display 2 2D images side-by-side

def display_2D_Images( fixed_npa, moving_npa ):
    # create a figure with two subplots and the specified size
    plt.subplots( 1, 2, figsize=( 10, 8 ) )
    
    # draw the fixed image in the first (left) subplot
    plt.subplot( 1, 2, 1 )
    plt.imshow( sitk.GetArrayViewFromImage(fixed_npa), cmap=plt.cm.Greys_r )
    plt.title( 'Fixed Image' )
    plt.axis( 'off' )
    
    # draw the moving image in the second (right) subplot
    plt.subplot( 1, 2, 2 )
    plt.imshow( sitk.GetArrayViewFromImage(moving_npa), cmap=plt.cm.Greys_r )
    plt.title( 'Moving Image' )
    plt.axis( 'off' )
    
    plt.show()

In [ ]:
# now, we visualize these images before registration
display_2D_Images( fixedImage, movingImage )

These two images are of the same image modality. Notice the position and the orientation of the head: in addition to the translation (the moving image is translated to the right, and down, compared to the fixed image), there is also a rotation of the head.

For the moment, we'll try to register these images using translation **only**.

Using the classical registration framework in SimpleITK, we need to specify the following components:
* a similarity metric,
* an optimizer,
* the type of transformation, and lastly
* a interpolator.


<img src="ITKv3_registration.png" width="850"/>

We have **not** discussed about interpolator, so for the moment we will ignore it.  The following code is the minimum implementation to perform registration in SimpleITK.

**Question**: Given what we know about these two images,
* What similarity metric would you use?
* What optimizer?

In [ ]:
registration_method = sitk.ImageRegistrationMethod()

# Similarity metric settings,
registration_method.SetMetricAsMeanSquares()

# interpolator
registration_method.SetInterpolator( sitk.sitkLinear )

# Optimizer setting
maxStep = 4.0
minStep = 0.01
numberOfIterations = 200
relaxationFactor = 0.5
registration_method.SetOptimizerAsRegularStepGradientDescent( maxStep, minStep, numberOfIterations, relaxationFactor )

# transform
initial_transform = sitk.TranslationTransform( fixedImage.GetDimension() )
registration_method.SetInitialTransform( initial_transform )

# Connect all of the observers so that we can perform plotting during registration.
registration_method.AddCommand(sitk.sitkStartEvent, rgui.start_plot)
registration_method.AddCommand(sitk.sitkEndEvent, rgui.end_plot)
registration_method.AddCommand(sitk.sitkMultiResolutionIterationEvent, rgui.update_multires_iterations) 
registration_method.AddCommand(sitk.sitkIterationEvent, lambda: rgui.plot_values(registration_method))


final_transform = registration_method.Execute( fixedImage, movingImage )

# Always check the reason optimization terminated.
print('Final metric value: {0}'.format(registration_method.GetMetricValue()))
print('Optimizer\'s stopping condition, {0}'.format(registration_method.GetOptimizerStopConditionDescription()))

To visualize the registration result, we will create a new image using the transform we computed from the registration, by applying the transform to the moving image:

In [ ]:
moving_resamples = sitk.Resample( movingImage, fixedImage, final_transform, sitk.sitkLinear, 0.0, movingImage.GetPixelID() )

display_2D_Images( fixedImage, moving_resamples )

In [ ]:
print(final_transform)

# if we are satisfied with the result, save the transform
sitk.WriteTransform( final_transform, os.path.join( OUTPUT_DIR, 'translation.tfm' ))

Note that the image is more-or-less aligned in terms of the position within the image, but the orientation of the head is still rotated.  We will fixe it in the next example.

<a href="AIP_Day_19_Transformation.ipynb"><h2 align=right>Next &raquo;</h2></a>